In [1]:
#get edge (no duplicated edge)
def getEdgeCount(line):
	u, v = line.split('\t')
	return ((u, v), 1)

def count(a, b):
	return a + b

def getEdge(line):
	edge, count = line
	return edge

#get out degree
def getUCount(edge):
	u, v = edge
	return (u, 1)

class weight(object):
	def __init__(self, uCount):
		self.uCount = dict(uCount)
		return

	def __call__(self, edge):
		u, v = edge
		return (u, (v, self.uCount[u]))

#a step of random walk
class trans(object):
	def __init__(self, r, beta = 0.8):
		self.r = dict(r)
		self.beta = beta
		self.nBeta = 1 - beta
		return

	def __call__(self, edgeW):
		u, vW = edgeW
		v, d = vW
		return (v, self.beta * (self.r[u] / d))

	def update(self, r):
		for v, val in r:
			self.r[v] = val + self.nBeta / 100
		return

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
842,application_1551670200070_0154,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
data = sc.textFile('/user/hz333/data/hw2/Q3/graph.txt')

#get edge (u, v)
line = data.map(getEdgeCount)
line = line.reduceByKey(count) #delete this line if all multiple edges are effective
edge = line.map(getEdge)

#get out degree
uCount = edge.map(getUCount)
uCount = uCount.reduceByKey(count)
uCount = uCount.collect()
getWeight = weight(uCount)

#get matrix (u, (v, degree))
edgeW = edge.map(getWeight)

#init r
r = [(str(i), 1 / 100) for i in range(1, 101)]
move = trans(r)

#loop
for i in range(40):
	r = edgeW.map(move)
	r = r.reduceByKey(count)
	r = r.collect()
	move.update(r)

In [3]:
r.sort(key = lambda vW: (vW[1], vW[0]), reverse = True)
for i in range(5):
	print('%s: %.5f' %(r[i][0], r[i][1]))
print('')

r.reverse()
for i in range(5):
	print('%s: %.5f' %(r[i][0], r[i][1]))
print('')

53: 0.03373
14: 0.03217
40: 0.03163
1: 0.02801
27: 0.02772

85: 0.00141
59: 0.00167
81: 0.00170
37: 0.00181
89: 0.00192